In [1]:
# TRANSFER LEARNING FINE TUNING -  MASS CLASSIFIER USING MAMMOGRAM PATCHES IN BCDR
import numpy as np
import tensorflow.keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, Convolution2D, Conv2D, MaxPooling2D, Lambda, GlobalMaxPooling2D, GlobalAveragePooling2D, BatchNormalization, Activation, AveragePooling2D, Concatenate
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
#from tensorflow.keras.utils import np_utils
%matplotlib inline
tensorflow.keras.backend.set_image_data_format('channels_last')
import tensorflow as tf
import os
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import Adam
from sklearn import metrics
from tensorflow.keras.layers import ReLU
import time


vid='v001_3'

In [2]:
# Create CNN

IMG_SIZE=224

from tensorflow.keras.applications.resnet50 import ResNet50
base_model=ResNet50(weights='imagenet',include_top=False, input_shape=(IMG_SIZE,IMG_SIZE,3))

for layer in base_model.layers:
    layer.trainable=False

x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(512)(x)
x=ReLU()(x)
x=Dropout(0.5)(x)
x=Dense(512)(x) 
x=ReLU()(x)
preds=Dense(1,activation='sigmoid')(x)

model=Model(inputs=base_model.input,outputs=preds)

In [3]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                              

                                                                                                  
 conv2_block3_1_relu (Activatio  (None, 56, 56, 64)  0           ['conv2_block3_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv2_block3_2_conv (Conv2D)   (None, 56, 56, 64)   36928       ['conv2_block3_1_relu[0][0]']    
                                                                                                  
 conv2_block3_2_bn (BatchNormal  (None, 56, 56, 64)  256         ['conv2_block3_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv2_block3_2_relu (Activatio  (None, 56, 56, 64)  0           ['conv2_block3_2_bn[0][0]']      
 n)       

                                                                                                  
 conv3_block3_1_relu (Activatio  (None, 28, 28, 128)  0          ['conv3_block3_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv3_block3_2_conv (Conv2D)   (None, 28, 28, 128)  147584      ['conv3_block3_1_relu[0][0]']    
                                                                                                  
 conv3_block3_2_bn (BatchNormal  (None, 28, 28, 128)  512        ['conv3_block3_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv3_block3_2_relu (Activatio  (None, 28, 28, 128)  0          ['conv3_block3_2_bn[0][0]']      
 n)       

                                                                                                  
 conv4_block2_1_bn (BatchNormal  (None, 14, 14, 256)  1024       ['conv4_block2_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv4_block2_1_relu (Activatio  (None, 14, 14, 256)  0          ['conv4_block2_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv4_block2_2_conv (Conv2D)   (None, 14, 14, 256)  590080      ['conv4_block2_1_relu[0][0]']    
                                                                                                  
 conv4_block2_2_bn (BatchNormal  (None, 14, 14, 256)  1024       ['conv4_block2_2_conv[0][0]']    
 ization) 

 conv4_block5_1_conv (Conv2D)   (None, 14, 14, 256)  262400      ['conv4_block4_out[0][0]']       
                                                                                                  
 conv4_block5_1_bn (BatchNormal  (None, 14, 14, 256)  1024       ['conv4_block5_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv4_block5_1_relu (Activatio  (None, 14, 14, 256)  0          ['conv4_block5_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv4_block5_2_conv (Conv2D)   (None, 14, 14, 256)  590080      ['conv4_block5_1_relu[0][0]']    
                                                                                                  
 conv4_blo

                                                                  'conv5_block1_3_bn[0][0]']      
                                                                                                  
 conv5_block1_out (Activation)  (None, 7, 7, 2048)   0           ['conv5_block1_add[0][0]']       
                                                                                                  
 conv5_block2_1_conv (Conv2D)   (None, 7, 7, 512)    1049088     ['conv5_block1_out[0][0]']       
                                                                                                  
 conv5_block2_1_bn (BatchNormal  (None, 7, 7, 512)   2048        ['conv5_block2_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv5_block2_1_relu (Activatio  (None, 7, 7, 512)   0           ['conv5_block2_1_bn[0][0]']      
 n)       

In [4]:
len(model.layers)

182

In [5]:
from tensorflow.keras import optimizers
model.compile(optimizer=optimizers.Adam(learning_rate=1e-3), loss='binary_crossentropy', metrics=['accuracy']) 

In [6]:
# Image preprocessing and data augmentation

batch_size=8

train_datagen = ImageDataGenerator(rescale=1./255,
                         horizontal_flip = True,
                         vertical_flip = True,
                         fill_mode="wrap",
                         width_shift_range = 0.2,
                         height_shift_range = 0.2,
                         zoom_range = 0.2,
                         rotation_range = 90,
                         shear_range=0.2
                        )

valid_datagen = ImageDataGenerator(rescale=1./255)

train_generator=train_datagen.flow_from_directory('mass_ben_mal_class02/train0'+vid[-1], 
                                                 target_size=(IMG_SIZE,IMG_SIZE),
                                                 color_mode='rgb',
                                                 batch_size=batch_size,
                                                 class_mode='binary',
                                                 shuffle=True)
valid_generator = valid_datagen.flow_from_directory('mass_ben_mal_class02/valid0'+vid[-1], 
                                                 target_size=(IMG_SIZE,IMG_SIZE),
                                                 color_mode='rgb',
                                                 batch_size=batch_size,
                                                 class_mode='binary',
                                                 shuffle=True)


Found 540 images belonging to 2 classes.
Found 126 images belonging to 2 classes.


In [7]:
#Callbacks
#checkpoint_path="resnet50_train4/cp.ckpt"
def get_callbacks(name_weights, patience_lr):
    mcp_save = ModelCheckpoint(name_weights, save_best_only=True, monitor='val_loss', mode='min')
    reduce_lr_loss = ReduceLROnPlateau(monitor='loss', factor=0.1, patience=patience_lr, verbose=2, min_delta=1e-4, mode='min')
    early_stop_cr=EarlyStopping(monitor='val_loss', min_delta=1e-4, patience=10, verbose=2, mode='auto', restore_best_weights=True)
    return [mcp_save, reduce_lr_loss, early_stop_cr]


In [8]:
#Training stage 1
step_size_train=train_generator.n//train_generator.batch_size
step_size_valid=valid_generator.n//valid_generator.batch_size

name_weights = "mass_ben_mal_class_resnet50_weights_"+vid+".h5"
callbacks = get_callbacks(name_weights = name_weights, patience_lr=10)

model.fit_generator(generator=train_generator,
                   steps_per_epoch=step_size_train,
                   validation_steps=step_size_valid,
                   validation_data=valid_generator,
                   epochs=3,
                   verbose=1,
                   callbacks = callbacks)  

C:\Users\Administrator\AppData\Local\Temp\ipykernel_19364\2463755422.py:8: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generator=train_generator,


Epoch 1/3
67/67 [==============================] - 67s 910ms/step - loss: 0.7643 - accuracy: 0.6034 - val_loss: 0.6463 - val_accuracy: 0.7250 - lr: 0.0010
Epoch 2/3
67/67 [==============================] - 59s 882ms/step - loss: 0.6613 - accuracy: 0.6316 - val_loss: 0.6104 - val_accuracy: 0.6917 - lr: 0.0010
Epoch 3/3
67/67 [==============================] - 58s 861ms/step - loss: 0.6606 - accuracy: 0.6447 - val_loss: 0.6174 - val_accuracy: 0.6917 - lr: 0.0010


In [9]:
#Training stage 2
for layer in model.layers[:120]:
    layer.trainable=False
for layer in model.layers[120:]:  #last ~33% layers are trainable
    layer.trainable=True
    
model.compile(optimizer=optimizers.Adam(learning_rate=1e-4), loss='binary_crossentropy', metrics=['accuracy'])

callbacks = get_callbacks(name_weights = name_weights, patience_lr=10)

model.fit_generator(generator=train_generator,
                   steps_per_epoch=step_size_train,
                   validation_steps=step_size_valid,
                   validation_data=valid_generator,
                   epochs=10,
                   verbose=1,
                   callbacks = callbacks)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_19364\3220737432.py:11: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generator=train_generator,


Epoch 1/10
67/67 [==============================] - 115s 2s/step - loss: 0.6515 - accuracy: 0.6541 - val_loss: 0.6045 - val_accuracy: 0.6917 - lr: 1.0000e-04
Epoch 2/10
67/67 [==============================] - 101s 2s/step - loss: 0.6030 - accuracy: 0.6823 - val_loss: 0.9136 - val_accuracy: 0.7083 - lr: 1.0000e-04
Epoch 3/10
67/67 [==============================] - 100s 1s/step - loss: 0.5776 - accuracy: 0.7256 - val_loss: 2.0634 - val_accuracy: 0.6750 - lr: 1.0000e-04
Epoch 4/10
67/67 [==============================] - 99s 1s/step - loss: 0.5784 - accuracy: 0.7312 - val_loss: 0.6247 - val_accuracy: 0.7083 - lr: 1.0000e-04
Epoch 5/10
67/67 [==============================] - 99s 1s/step - loss: 0.6204 - accuracy: 0.7068 - val_loss: 0.6303 - val_accuracy: 0.6833 - lr: 1.0000e-04
Epoch 6/10
67/67 [==============================] - 98s 1s/step - loss: 0.5666 - accuracy: 0.7124 - val_loss: 1.5227 - val_accuracy: 0.7000 - lr: 1.0000e-04
Epoch 7/10
67/67 [==============================] - 98s

In [10]:
#Training stage 3
#time.sleep(360)
for layer in model.layers:  #All layers are trainable
    layer.trainable=True
    
model.compile(optimizer=optimizers.Adam(learning_rate=1e-5), loss='binary_crossentropy', metrics=['accuracy'])
callbacks = get_callbacks(name_weights = name_weights, patience_lr=5)

model.fit_generator(generator=train_generator,
                   steps_per_epoch=step_size_train,
                   validation_steps=step_size_valid,
                   validation_data=valid_generator,
                   epochs=150,
                   verbose=1,
                   callbacks = callbacks)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_19364\2279829289.py:9: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generator=train_generator,


Epoch 1/150
67/67 [==============================] - 231s 3s/step - loss: 0.5495 - accuracy: 0.7293 - val_loss: 7.4773 - val_accuracy: 0.6917 - lr: 1.0000e-05
Epoch 2/150
67/67 [==============================] - 203s 3s/step - loss: 0.5026 - accuracy: 0.7970 - val_loss: 0.6045 - val_accuracy: 0.7083 - lr: 1.0000e-05
Epoch 3/150
67/67 [==============================] - 198s 3s/step - loss: 0.4529 - accuracy: 0.8120 - val_loss: 0.6321 - val_accuracy: 0.7083 - lr: 1.0000e-05
Epoch 4/150
67/67 [==============================] - 197s 3s/step - loss: 0.4080 - accuracy: 0.8365 - val_loss: 0.7291 - val_accuracy: 0.7083 - lr: 1.0000e-05
Epoch 5/150
67/67 [==============================] - 199s 3s/step - loss: 0.3874 - accuracy: 0.8365 - val_loss: 0.6360 - val_accuracy: 0.6833 - lr: 1.0000e-05
Epoch 6/150
67/67 [==============================] - 201s 3s/step - loss: 0.3950 - accuracy: 0.8365 - val_loss: 0.5618 - val_accuracy: 0.6917 - lr: 1.0000e-05
Epoch 7/150
67/67 [===========================

In [11]:
model.metrics_names

['loss', 'accuracy']

In [12]:
# Validation accuracy, validation data confusion matrix, area under the roc score
valid_generator = valid_datagen.flow_from_directory('mass_ben_mal_class02/valid0'+vid[-1], 
                                                 target_size=(IMG_SIZE,IMG_SIZE),
                                                 color_mode='rgb',
                                                 batch_size=batch_size,
                                                 class_mode='binary',
                                                 shuffle=False)

# Ref: https://gist.github.com/RyanAkilos/3808c17f79e77c4117de35aa68447045 accessed on 5 Feb 2020
from sklearn.metrics import precision_score, recall_score, confusion_matrix, accuracy_score, roc_auc_score, f1_score, matthews_corrcoef


Y_pred = model.predict_generator(valid_generator)
y_pred=(Y_pred>0.5).astype(int)
ras=roc_auc_score(valid_generator.classes, Y_pred)
prec=precision_score(valid_generator.classes, y_pred,pos_label=1)
rec=recall_score(valid_generator.classes, y_pred,pos_label=1)
spe=recall_score(valid_generator.classes, y_pred,pos_label=0)
f1s=f1_score(valid_generator.classes, y_pred,pos_label=1)
mcc=matthews_corrcoef(valid_generator.classes, y_pred)

print('Confusion Matrix:')
print(confusion_matrix(valid_generator.classes, y_pred))

print('Sensitivity: ', rec)
print('Specificity: ', spe)
print('Precision score:', prec)
print('Accuracy:', accuracy_score(valid_generator.classes, y_pred))
print('F1 score:', f1s)
print('MCC score:', mcc)
print('ROC AUC score:', ras)


Found 126 images belonging to 2 classes.


C:\Users\Administrator\AppData\Local\Temp\ipykernel_19364\1624802210.py:13: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  Y_pred = model.predict_generator(valid_generator)


Confusion Matrix:
[[78  9]
 [10 29]]
Sensitivity:  0.7435897435897436
Specificity:  0.896551724137931
Precision score: 0.7631578947368421
Accuracy: 0.8492063492063492
F1 score: 0.7532467532467534
MCC score: 0.6448144432620804
ROC AUC score: 0.9277925139994105


In [13]:
train_generator.filenames

['ben\\0101e9d3-ea0f-46ff-a007-dc0e95594f74_987_overlay_patch.png',
 'ben\\024dc580-4d90-45a8-92e7-78b79fb439eb_53_overlay_patch.png',
 'ben\\025ed8da-6106-4e97-b8cd-bdb2259c9740_407_overlay_patch.png',
 'ben\\035ff673-85f8-4b71-b054-e2b9e0a9e43c_1271_overlay_patch.png',
 'ben\\03a6fb93-c2fe-427e-b2a6-5824092c1049_169_overlay_patch.png',
 'ben\\04a9fdf8-a513-495b-8454-1c7d0e48fb25_193_overlay_patch.png',
 'ben\\0622661b-972c-4f20-91bf-bdd32c9322e3_974_overlay_patch.png',
 'ben\\07fd4591-b192-4ddd-ada9-036049138557_1177_overlay_patch.png',
 'ben\\07fd4591-b192-4ddd-ada9-036049138557_172_overlay_patch.png',
 'ben\\0830491b-2783-43a2-ab94-8e479fd45067_933_overlay_patch.png',
 'ben\\08791a46-45bd-469f-93b7-f6876f9cad29_52_overlay_patch.png',
 'ben\\08a39649-560e-4744-8197-2910bf19ae02_1176_overlay_patch.png',
 'ben\\08a39649-560e-4744-8197-2910bf19ae02_173_overlay_patch.png',
 'ben\\0c069e44-e299-4d92-8069-35645ca98f2f_795_overlay_patch.png',
 'ben\\0c5d8e4b-be54-434b-bd92-b08bfd217c71_768

In [14]:
valid_generator.filenames

['ben\\0038f5af-6350-403a-9924-342a870c808a_608_overlay_patch.png',
 'ben\\0101e9d3-ea0f-46ff-a007-dc0e95594f74_984_overlay_patch.png',
 'ben\\0215715c-de35-4dfc-b613-14caf4fef059_718_overlay_patch.png',
 'ben\\0918eb8b-8720-4dec-a655-8a293ce7bc5c_1482_overlay_patch.png',
 'ben\\09cfba2d-00bf-4199-80e0-6dcfb2a089fc_1347_overlay_patch.png',
 'ben\\0f02a8d0-de6a-4203-b00b-a1b26ef3fbb0_268_overlay_patch.png',
 'ben\\12110be6-87a8-405c-98ad-17fce5dc14b2_1361_overlay_patch.png',
 'ben\\169ed1c9-230b-42b3-88f5-7914cc525ad3_1079_overlay_patch.png',
 'ben\\1b719214-eba7-4d36-80c0-2b34fc61cd47_1044_overlay_patch.png',
 'ben\\20e414b9-5e0b-4924-a595-70bb4de85eed_1470_overlay_patch.png',
 'ben\\213046fd-24f7-4cf5-a7cf-c87b2b343942_734_overlay_patch.png',
 'ben\\277e3276-eecf-4dbd-a23b-a2fa9ab4e425_1119_overlay_patch.png',
 'ben\\2a6aa030-41ca-484c-b3e9-2194f870f506_1216_overlay_patch.png',
 'ben\\2e078c9c-e262-4aef-9e47-c1a2b98e132e_206_overlay_patch.png',
 'ben\\2ff4b5c1-1b69-418b-864e-1b6f8b6f2

In [15]:
#Print estimation result for ensemble model
print(Y_pred)
print(valid_generator.classes)

[[1.1047564e-02]
 [1.3746144e-01]
 [1.1889322e-02]
 [1.5011336e-02]
 [7.4851641e-04]
 [2.7250370e-01]
 [4.3602176e-02]
 [3.9007247e-03]
 [4.3836463e-02]
 [1.8681623e-01]
 [7.6477051e-02]
 [8.4021442e-02]
 [1.2500416e-01]
 [8.4925562e-01]
 [5.4049021e-01]
 [1.0986917e-02]
 [4.8711300e-01]
 [1.4692014e-01]
 [8.6836498e-03]
 [2.2171456e-02]
 [1.1823935e-02]
 [7.4896388e-02]
 [2.7846882e-02]
 [1.1714603e-01]
 [3.9515777e-05]
 [6.3872731e-01]
 [2.7465168e-03]
 [1.6475337e-03]
 [2.6288694e-01]
 [8.6004071e-02]
 [1.6162179e-01]
 [2.4977948e-01]
 [2.9501094e-02]
 [7.3394668e-03]
 [4.4692537e-01]
 [8.4662640e-01]
 [3.7561934e-02]
 [1.3932951e-01]
 [1.4049618e-02]
 [1.1846159e-02]
 [3.1653938e-06]
 [1.0597924e-02]
 [1.6630413e-01]
 [3.6204006e-03]
 [3.2315392e-02]
 [5.4875854e-03]
 [2.9053853e-03]
 [2.4555800e-03]
 [5.8915919e-01]
 [8.2882404e-02]
 [1.5462575e-03]
 [4.2563085e-03]
 [2.0166615e-03]
 [2.9806131e-02]
 [2.7480070e-02]
 [2.8361145e-01]
 [1.9458073e-01]
 [7.8627831e-01]
 [2.0228449e-0

In [16]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory('mass_ben_mal_class02/test', 
                                                 target_size=(IMG_SIZE,IMG_SIZE),
                                                 color_mode='rgb',
                                                 batch_size=batch_size,
                                                 class_mode='binary',
                                                 shuffle=False)


# Ref: https://gist.github.com/RyanAkilos/3808c17f79e77c4117de35aa68447045 accessed on 5 Feb 2020
from sklearn.metrics import precision_score, recall_score, confusion_matrix, accuracy_score, roc_auc_score, f1_score, matthews_corrcoef


Y_pred = model.predict_generator(test_generator)
y_pred=(Y_pred>0.5).astype(int)
ras=roc_auc_score(test_generator.classes, Y_pred)
prec=precision_score(test_generator.classes, y_pred,pos_label=1)
rec=recall_score(test_generator.classes, y_pred,pos_label=1)
spe=recall_score(test_generator.classes, y_pred,pos_label=0)
f1s=f1_score(test_generator.classes, y_pred,pos_label=1)
mcc=matthews_corrcoef(test_generator.classes, y_pred)

print('Confusion Matrix:')
print(confusion_matrix(test_generator.classes, y_pred))

print('Sensitivity: ', rec)
print('Specificity: ', spe)
print('Precision score:', prec)
print('Accuracy:', accuracy_score(test_generator.classes, y_pred))
print('F1 score:', f1s)
print('MCC score:', mcc)
print('ROC AUC score:', ras)


Found 169 images belonging to 2 classes.


C:\Users\Administrator\AppData\Local\Temp\ipykernel_19364\1073823509.py:15: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  Y_pred = model.predict_generator(test_generator)


Confusion Matrix:
[[103  10]
 [ 17  39]]
Sensitivity:  0.6964285714285714
Specificity:  0.911504424778761
Precision score: 0.7959183673469388
Accuracy: 0.8402366863905325
F1 score: 0.7428571428571428
MCC score: 0.6306672630383637
ROC AUC score: 0.8663084702907712


In [17]:
test_generator.filenames

['ben\\03a24bd4-74c0-485b-b1f0-eb9f2c402935_1352_overlay_patch.png',
 'ben\\05e1bfd9-3786-40d7-8b4e-2b9245ea929c_1006_overlay_patch.png',
 'ben\\070697d4-5446-4514-ac7e-3d665efcfb67_1171_overlay_patch.png',
 'ben\\083609df-1ff7-4a38-8be4-083bd05544e2_290_overlay_patch.png',
 'ben\\0cecfab0-f689-4947-8821-fbb2138027d7_435_overlay_patch.png',
 'ben\\0cecfab0-f689-4947-8821-fbb2138027d7_438_overlay_patch.png',
 'ben\\0f55e315-6055-4ad7-bd22-e0036e364ba7_716_overlay_patch.png',
 'ben\\1031a46b-cd0e-46bf-bac1-71515c66df0f_1338_overlay_patch.png',
 'ben\\12110be6-87a8-405c-98ad-17fce5dc14b2_1363_overlay_patch.png',
 'ben\\17e65a60-682b-4395-bcc1-da6d25a69f61_436_overlay_patch.png',
 'ben\\17e65a60-682b-4395-bcc1-da6d25a69f61_437_overlay_patch.png',
 'ben\\18efb98a-c54d-4cde-bc72-16ab05e12fe6_951_overlay_patch.png',
 'ben\\19363b96-1a67-4c6c-a172-29e03f523c27_1125_overlay_patch.png',
 'ben\\1b6f6632-b6d8-4d5c-ba0c-a732e7ddc24a_950_overlay_patch.png',
 'ben\\1cafc30d-b791-42e7-b687-24233fc299e

In [18]:
#Print estimation result for ensemble model
print(Y_pred)
print(test_generator.classes)

[[9.07846987e-02]
 [6.45926476e-01]
 [6.48346692e-02]
 [1.83160063e-02]
 [5.72052747e-02]
 [6.74965829e-02]
 [3.09285708e-02]
 [1.08047817e-02]
 [5.18092960e-02]
 [1.74955279e-02]
 [1.39614921e-02]
 [8.21167976e-02]
 [3.01674455e-02]
 [1.26162723e-01]
 [6.45263167e-03]
 [6.96576308e-07]
 [8.06299508e-01]
 [9.07392561e-01]
 [1.62383914e-02]
 [4.64721099e-02]
 [3.41417432e-01]
 [5.21266786e-03]
 [8.43105912e-02]
 [1.41514793e-01]
 [5.57700871e-04]
 [2.71999817e-02]
 [9.72614251e-03]
 [1.52580254e-02]
 [8.91873017e-02]
 [5.24685606e-02]
 [5.35510422e-04]
 [2.53694236e-01]
 [1.46690030e-02]
 [5.00069000e-02]
 [2.53633469e-01]
 [1.38270645e-03]
 [1.07988894e-01]
 [1.56381488e-01]
 [1.87579531e-03]
 [8.98683488e-01]
 [1.49193197e-01]
 [5.82203455e-03]
 [1.50954677e-02]
 [9.53214243e-03]
 [4.74892482e-02]
 [8.30899365e-03]
 [5.93560282e-03]
 [2.10421291e-04]
 [7.13457406e-01]
 [2.07961481e-02]
 [6.20462000e-02]
 [4.63528335e-02]
 [2.57042255e-02]
 [5.89127839e-03]
 [2.73635983e-01]
 [5.906069